# Cuaderno 8: Calendarización de trabajos (job scheduling)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$

Los problemas de calendarización de trabajos (*job scheduling*) consisten en determinar el orden óptimo en el que deben ejecutarse ciertas tareas o trabajos, en una o más máquinas, para optimizar un proceso productivo.

Suponer que se tiene dado un conjunto de $n$ trabajos de confección $J:= \tabulatedset{1, \ldots, n}$, los cuales  deben ser procesados secuencialmente en una máquina hiladora. La máquina contiene un carrete en el cual pueden ser cargados simultáneamente $B$ hilos de diferentes colores, seleccionados de un conjunto $H$ de hilos disponibles. Cada trabajo $j \in J$  requiere para su ejecución de un subconjunto $H_j \subset H$ de hilos. Para que $j$
pueda ser procesado, todos los hilos de $H_j$ deben estar cargados en el carrete. Si esto no ocurre, la máquina debe ser detenida para cargar los hilos que hagan falta y, de ser necesario, descargar hilos que no sean requeridos con la finalidad de liberar espacio en el carrete. Asumimos que $\card{H_j} \leq B, \forall j \in J$.

Se desea determinar el orden en el que los trabajos deben ser procesados para minimizar el número de paradas requeridas de la máquina. Para especificar el ordenamiento, se asignará a cada trabajo $j \in J$ un turno $t \in T$, con $T:= \tabulatedset{1, \ldots, n}$. 

Emplearemos las siguientes variables de decisión binarias: 

* $x_{jt}$, $j \in J$, $t \in T$, que indican si el trabajo $j$ es procesado en el turno $t$;
* $y_{ht}$, $h \in H$, $t \in T$, que determinan si el hilo $h \in H$ está cargado en la máquina durante el turno $t$; 
* $z_t$, $t \in T, \, t \geq 2$, que registran si es necesario parar la máquina entre los turnos $t-1$ y $t$.

Los valores de $z_t$ se determinan a partir de los valores de $y_{ht}$ como se indica a continuación. Notar que la expresión es $\card{y_{ht} - y_{h,t-1}}$ es igual a 1 si y sólo si el hilo $h$ entra o sale del carrete de la máquina entre los turnos $t-1$ y $t$. Además, definiendo:
\begin{align*}
w_{ht} \in & \{0, 1 \} \, : \\
& w_{ht} \geq y_{ht} - y_{h,t-1}, \\
& w_{ht} \geq y_{h,t-1} - y_{ht},
\end{align*}
tenemos que $w_{ht} \geq \card{y_{ht} - y_{h,t-1}}$. Notar que $\sum_{h \in H} w_{ht}$ es forzosamente mayor que cero si *por lo menos un* hilo entra o sale del carrete entre los turnos $t-1$ y $t$, es decir, si es necesario parar la máquina entre estos turnos. Por lo tanto, si tomamos $z_{t} \in \{0, 1 \}$, con $z_t \geq \frac{\sum_{h \in H} w_{ht}}{\card{H}}$, se tiene que $z_t$ debe valer $1$ si es necesario detener la máquina entre los turnos $t-1$ y $t$ para cambiar algún hilo.

Empleando estas ideas, el problema de calendarización de trabajos descrito arriba puede ser formulado como el siguiente modelo de programación lineal entera.  

\begin{align*}
\min &\sum_{t \in T \setminus \{1\}} z_t\\ 
& \mbox{s.r.}\\
&\sum_{j \in J} x_{jt} = 1, \quad \forall t \in T, \\
&\sum_{t \in T} x_{jt} = 1, \quad \forall j \in J, \\
&\sum_{h \in H} y_{ht} \leq B, \quad \forall t \in T, \\
& x_{jt} \leq y_{ht}, \quad \forall h \in H_j, \, j \in J, \, t \in T,\\
& w_{ht} \geq y_{ht} - y_{h, t-1}, \quad, \forall h \in H, t \in T \setminus \{1\},\\
& w_{ht} \geq y_{h, t-1} - y_{ht}, \quad, \forall h \in H, t \in T \setminus \{1\},\\
& \sum_{h \in H} w_{ht} \leq \card{H} z_t, \forall t \in T \setminus \{1\},\\
& x_{jt}, y_{ht} \in \{0, 1\}, \quad \forall j \in J, h \in H, t \in T,\\
& w_{ht}, z_t \in \{0, 1\}, \quad \forall h \in H, t \in T \setminus \{1\}.
\end{align*}

La función objetivo mide la cantidad de turnos $t \in T \setminus \{1\}$ para los cuales $z_t=1$. De lo expuesto arriba, forzosamente esta cantidad es, por lo menos, la cantidad de paradas de la máquina dentro de la solución. Por otra parte, como se trata de un problema de minimización, puede demostrarse que, en toda solución óptima, el valor de la función objetivo coincide con el número de paradas de la máquina.

Las dos primeras familias de restricciones son restricciones de asignación entre los trabajos y los turnos: cada trabajo debe ser asignado a un turno, y cada turno debe tener un trabajo asignado.

La tercera familia de restricciones indica que, en cada turno, la cantidad de hilos cargados en el carrete no puede exceder su capacidad.

La cuarta familia de restricciones establece que, para que un trabajo sea procesado, todos los hilos requeridos por el mismo deben estar cargados.

Las últimas tres familias de restricciones ajustan el valor de $z_t$ como se explicó arriba, con la finalidad de que $z_t = 1$ cada vez que es necesario parar la máquina entre los turnos $t-1$ y $t$.


Vamos a implementar este modelo usando la interfaz Python de Gurobi.


Definimos primero los conjuntos $J$, $H$, $T$; y los parámetros $H_j$ y $B$:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# import random

# Conjuntos y parámetros del modelo
# Familia HH de conjuntos de hilos requeridos por cada trabajo
HH = gp.tupledict({1 : {2, 3, 5},
                2 : {1, 3, 4},
                3 : {1, 4, 5},
                4 : {1, 3},
                5 : {1, 2, 4},
                6 : {1, 4, 5},
                7 : {3, 5}})

# Cantidad de hilos que pueden estar en el carrete
B = 3

# ---- a partir de aqui, los parametros se calculan en base a los anteriores ----

# Conjunto de trabajos
J = HH.keys()

# Conjunto de turnos T y de turnos sin el primero
T = J.copy()
T2 = [i for i in T if i!=1]

# Conjunto de todos los hilos, calculado como la union de Hj para j in J
H = set()
for j in J:
    H = H | HH[j]

print("H = {}".format(H))
print("J = {}".format(J))
print("T = {}".format(T))
print("HH = {}".format(HH))


Definimos ahora el objeto modelo y las variables del modelo:

In [ ]:
m = gp.Model('job-scheduling')

# Asignacion turnos a trabajos
x = m.addVars(J, T, vtype = GRB.BINARY, name="x")

# Estado de hilos en el carrete
y = m.addVars(H, T, vtype = GRB.BINARY, name="y")

# Paradas de la maquina
z = m.addVars(T2, vtype = GRB.BINARY, name="z")

# Auxiliar: cambio de los hilos en el carrete
w = m.addVars(H, T2, vtype = GRB.BINARY, name="w")

Definimos la función objetivo:

In [ ]:
# minimizar paradas de la maquina
m.setObjective(z.sum('*'), GRB.MINIMIZE)

Finalmente, implementamos las restricciones del modelo. El método `addConstrs` nos permite agregar una familia completa de restricciones al modelo, empleando un generador.

1. A cada trabajo $j$ se le asigna un turno único y en cada turno se ejecuta exactamente un trabajo.

In [ ]:
# A cada turno se le asigna exactamente un trabajo
m.addConstrs((x.sum('*', t) == 1 for t in T), name='turno_t') 

# A cada trabajo se le asigna exactamente un turno
m.addConstrs((x.sum(j, '*') == 1 for j in J), name='turno_j') 

2. En cada turno, la cantidad de hilos cargados en la máquina no supera la capacidad del carrete.

In [ ]:
# Respetar capacidad del carrete
m.addConstrs((y.sum('*', t) <= B for t in T), name='carrete') 

3. Si el trabajo $j$ es procesado en el turno $t$, entonces todos los hilos requeridos para este trabajo deben estar cargados en el carrete en el turno $t$.

In [ ]:
# Cargar todos los hilos requeridos para procesar un trabajo
m.addConstrs((x[j,t] <= y[h,t] for t in T for j in J for h in HH[j]), name='hilos') 

4. Fijamos el valor de las variables auxiliares $w_{ht}$:

In [ ]:
# Definir w[h,t] en función de y[h,t] y y[h,t-1]
m.addConstrs((w[h,t] >= y[h,t] - y[h,t-1] for h in H for t in T2), name='defw1') 
m.addConstrs((w[h,t] >= y[h,t-1] - y[h,t] for h in H for t in T2), name='defw2') 

5. Finalmente, fijamos el valor de $z_t$ de tal forma que $z_t=1$ cada vez que sea necesario para la máquina entre los turnos $t-1$ y $t$:

In [ ]:
# Definir z[t] en función de w[h,t]
m.addConstrs((w.sum('*', t) <= len(H)*z[t] for t in T2), name='defz') 

Optimizamos el modelo:

In [ ]:
m.optimize()

Mostramos la solución: instalaciones a construir, asignación de clientes a instalaciones y clientes no atendidos.

In [ ]:
# Mostrar solución
print('Orden de procesamiento de trabajos:')
print([j for t in T for j in J if x[j,t].x >= 0.9])

print('Numero de paradas: {}'.format(m.objVal))

print('Estado del carrete:')
for t in T:
    print('{}: {}'.format(t, [h for h in H if y[h,t].x >= 0.9]))

print('Hilos requeridos por trabajo')
for j in J:
    print('{}: {}'.format(j, HH[j]))

## Configurando parámetros del solver

El comportamiento del solver Gurobi puede ser modificado a través de la fijación de ciertos *parámetros del solver*. En el API Python el acceso a estos parámetros puede realizarse a través de la propiedad `Params` en el objeto modelo.  

Dos parámetros especialmente importantes son [`TimeLimit`](https://www.gurobi.com/documentation/9.5/refman/timelimit.html), que controla el *tiempo límite* disponible para la solución de un modelo (en segundos), y [`MIPGap`](https://www.gurobi.com/documentation/9.5/refman/mipgap2.html), que indica la *brecha de optimalidad aceptable* para declarar a una solución de un programa entero como solución óptima (como fracción de la unidad). Estos parámetros resultan útiles al trabajar con modelos de gran escala que demandan demasiado tiempo de ejecución. 

Para mayor información acerca de los parámetros de Gurobi, consultar en el siguiente [enlace](https://www.gurobi.com/documentation/9.5/refman/parameters.html).

In [ ]:
# Fijar tiempo máximo de ejecución en 60 segundos
m.Params.TimeLimit = 60

# Fijar brecha de optimalidad aceptable en 10%
m.Params.MIPGap = 0.1

Al fijar tiempos límites para la ejecución, a veces es necesario poder determinar si durante el último llamado a `optimize()` fue posible encontrar al menos una solución factible para el modelo. Esto puede hacerse consultando el atributo `SolCount` de la clase modelo, el cual almacena el número de soluciones factibles encontradas.

In [ ]:
# Indicar el número de solucione factibles encontradas
print(m.SolCount)

## Código completo

Reproducimos a continuación el código completo del ejemplo:

In [ ]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de job scheduling
# EPN (2022)

import gurobipy as gp
from gurobipy import GRB
import random as rd

try:
    # Conjuntos y parametros del modelo
    # Familia HH de conjuntos de hilos requeridos por cada trabajo
    # HH = tupledict({1 : {2, 3, 5},
    #                  2 : {1, 3, 4},
    #                  3 : {1, 4, 5},
    #                  4 : {1, 3},
    #                  5 : {1, 2, 4},
    #                  6 : {1, 4, 5},
    #                  7 : {3, 5}})
    
    # Generar n trabajos al azar, cada uno requiere de 1 a tres hilos
    # La cantidad total de hilos es n_hilos
    n_hilos = 10
    n = 100
    HH = gp.tupledict({j+1 : set([rd.randint(1,n_hilos) for i in range(3)]) for j in range(n)})

    # HH = {j : set([random.randint(1,5) for i in range(3)]) for j in range(1,101)}
    # print("HH= {}".format(HH))

    # Cantidad de hilos que pueden estar en el carrete
    B = 3

    # Conjunto de trabajos
    J = HH.keys()

    # Conjunto de turnos T y de turnos T2 sin el primero
    T = J
    T2 = [i for i in T if i!=1]

    # Conjunto de todos los hilos, calculado como la union de Hj para j in J
    H = set()
    for j in J:
        H = H | HH[j]
    
    # Crear el objeto modelo
    m = gp.Model('job-scheduling')

    # Definir variables
    # Asignación turnos a trabajos
    x = m.addVars(J, T, vtype = GRB.BINARY, name="x")

    # Estado de hilos en el carrete
    y = m.addVars(H, T, vtype = GRB.BINARY, name="y")

    # Paradas de la máquina
    z = m.addVars(T2, vtype = GRB.BINARY, name="z")

    # Cambio de los hilos en el carrete
    w = m.addVars(H, T2, vtype = GRB.BINARY, name="z")
    
    # Función objetivo: minimizar paradas de la máquina
    m.setObjective(z.sum('*'), GRB.MINIMIZE)

    # Restricciones
    # A cada turno se le asigna exactamente un trabajo
    m.addConstrs((x.sum('*', t) == 1 for t in T), name='turno_t') 

    # A cada trabajo se le asigna exactamente un turno
    m.addConstrs((x.sum(j, '*') == 1 for j in J), name='turno_j') 
    
    # Respetar capacidad del carrete
    m.addConstrs((y.sum('*', t) <= B for t in T), name='carrete') 
    
    # Cargar todos los hilos requeridos para procesar un trabajo
    m.addConstrs((x[j,t] <= y[h,t] for t in T for j in J for h in HH[j]), name='hilos') 
    
    # Definir w[h,t] en función de y[h,t] y y[h,t-1]
    m.addConstrs((w[h,t] >= y[h,t] - y[h,t-1] for h in H for t in T2), name='defw1') 
    m.addConstrs((w[h,t] >= y[h,t-1] - y[h,t] for h in H for t in T2), name='defw2') 
    
    # Definir z[t] en función de w[h,t]
    m.addConstrs((w.sum('*', t) <= len(H)*z[t] for t in T2), name='defz') 
    
    # Fijar tiempo máximo de ejecución en 90 segundos
    m.Params.TimeLimit = 90

    # Fijar brecha de optimalidad aceptable en 5%
    m.Params.MIPGap = 0.05

    # Resolver el modelo
    m.optimize()
    
    # Mostrar solución
    
    # Para saber si existe al menos una solución, consultar el atributo SolCount de la clase modelo
    if m.SolCount > 0:
        print('Orden de procesamiento de trabajos:')
        print([j for t in T for j in J if x[j,t].x >= 0.9])

        print('Número de paradas: {}'.format(m.objVal))

        print('Estado del carrete:')
        for t in T:
            print('{}: {}'.format(t, [h for h in H if y[h,t].x >= 0.9]))
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')